In [1]:
import pandas as pd
from googleapiclient.discovery import build

In [2]:
df = pd.read_csv('./channel_review.csv')
df.head()

,CHANNEL_TITLE,TAGS,CHANNEL_ID,LR,RELEVANCE,REVIEWER_LR,REVIEWER_TAGS,REVIEWER_MAIN_CHANNEL_ID,REVIEWER_RELEVANCE,REVIEWER_CODE,UPDATED,PUBLIC_REVIEWER_NOTES,PUBLIC_CREATOR_NOTES
0,Brave,NaN,UCFNTTISby1c_H-rm5Ww5rZg,C,0.0,C,NaN,NaN,0.0,mar,2021-12-22T01:00:52.5691836Z,NaN,NaN
1,GBNews,AntiSJW|Mainstream News|PartisanRight,UC0vn8ISa4LKMunLbzaXLnOQ,R,1.0,R,AntiSJW|Mainstream News|PartisanRight,NaN,1.0,mar,2021-10-25T21:17:49.7200458Z,NaN,NaN
2,The Black Conservative Preacher,AntiSJW|PartisanRight|QAnon,UC_QYq3wFd46rcaoKOvPHFgA,R,1.0,R,PartisanRight|QAnon|AntiSJW,NaN,1.0,mar,2021-06-08T01:27:45.4527338Z,This is the channel that had the senator who l...,NaN
3,Energy and Commerce Committee,StateFunded,UCCbD3bkHRcwiBsaL1lWE_QQ,C,1.0,C,StateFunded,NaN,1.0,mar,2021-03-26T02:15:09.3585105Z,NaN,NaN
4,That Star Wars Girl,AntiSJW,UCRWsFMLzGGJKLnAXJ9U93_A,R,0.8,R,AntiSJW,NaN,0.8,mar,2021-03-16T01:24:18.9299734Z,NaN,NaN


In [10]:
# Must contain 'AntiTheist' as a label from the TAGS column, NOT the TAGS_REVIEWER column 
#   (that is before conslidating labels via their inter-coder agreement rule)
# Also cannot be a BitChute channel 
#   (in the bottom of the paper they mention that some channels migrate to other platform, such as BitChute)
df['is_conspiracy'] = df.apply(
    lambda row: 'Conspiracy' in str(row['TAGS']) and str(row['CHANNEL_ID'])[:2]=='UC',
    axis = 1
)

In [11]:
df_conspiracy = df[df['is_conspiracy']]
df_conspiracy.head()

,CHANNEL_TITLE,TAGS,CHANNEL_ID,LR,RELEVANCE,REVIEWER_LR,REVIEWER_TAGS,REVIEWER_MAIN_CHANNEL_ID,REVIEWER_RELEVANCE,REVIEWER_CODE,UPDATED,PUBLIC_REVIEWER_NOTES,PUBLIC_CREATOR_NOTES,is_conspiracy
12,Road To Freedom,Conspiracy,UCsHaTGHXK1B0GrInsGWOmGg,R,0.7,R,Conspiracy,NaN,0.7,scl,2021-02-12T19:34:04.2286870Z,NaN,NaN,True
16,BeachBroadcast,Conspiracy|PartisanRight|QAnon,UCdV9pD5PKmSG3rgTA9Ld5rg,R,1.0,R,PartisanRight|QAnon|Conspiracy,UC0v7VUtxrPnQWPgZx-WSzgw,1.0,mar,2021-02-12T03:03:30.9507915Z,NaN,NaN,True
17,Santa Surfing Beach Broadcast,Conspiracy|PartisanRight|QAnon,UC0v7VUtxrPnQWPgZx-WSzgw,R,1.0,R,Conspiracy|QAnon|PartisanRight,NaN,1.0,mar,2021-02-12T03:02:57.4433881Z,NaN,NaN,True
112,NaturalNews,Conspiracy|PartisanRight|QAnon,UCdRRA3aC-e4Njs2q5jOVTCw,R,1.0,R,Conspiracy|PartisanRight|QAnon,NaN,1.0,mar,2021-02-10T02:59:05.7567176Z,NaN,NaN,True
121,The Dollar Vigilante,Conspiracy|PartisanRight,UCpf2Lxgf10AjzBT4GIIgDJw,R,1.0,R,PartisanRight|Conspiracy,NaN,1.0,mar,2021-02-10T02:30:30.2084380Z,NaN,NaN,True


In [16]:
df_conspiracy.shape

(2820, 14)

In [17]:
df_conspiracy['CHANNEL_ID'].unique()

array(['UCsHaTGHXK1B0GrInsGWOmGg', 'UCdV9pD5PKmSG3rgTA9Ld5rg',
       'UC0v7VUtxrPnQWPgZx-WSzgw', ..., 'UCBpSlvedUI6VmwBjQLpFb0g',
       'UCrQ-wHKVi0JDWjQGcuoYnew', 'UCyQsQSKayfcZoHaCjqxxSlg'],
      dtype=object)

In [18]:
# From old build_polarizer (see Alex branch of Siqi's Polarizer code)
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
dev_key_file = '../yt_api_dev_key.txt'
text_file = open(dev_key_file, "r")
DEVELOPER_KEY = text_file.read()
text_file.close()

In [20]:
def check_api_available(cid):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=cid
    )
    response = request.execute()
    if response['pageInfo']['totalResults'] == 0:
        return False
    return True


In [21]:
ids = []
for cid in df_conspiracy['CHANNEL_ID']:
    if check_api_available(cid):
        ids.append(cid)


In [22]:
d = {'id':ids}
df = pd.DataFrame(d)

In [23]:
df.to_csv('conspiracy.csv', index=False, header=False)